# Crossvalidation and Regularization

Peter Ralph

https://uodsci.github.io/dsci345

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15, 8)
import numpy as np
import pandas as pd
from dsci345 import pretty

rng = np.random.default_rng(123)

$$\renewcommand{\P}{\mathbb{P}} \newcommand{\E}{\mathbb{E}} \newcommand{\var}{\text{var}} \newcommand{\sd}{\text{sd}} \newcommand{\cov}{\text{cov}} \newcommand{\cor}{\text{cor}}$$
This is here so we can use `\P` and `\E` and `\var` and `\cov` and `\cor` and `\sd` in LaTeX below.

# A cautionary tale

In [ ]:
import datetime
full_covid = pd.read_csv("data/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")[["state", "tot_cases", "submission_date"]]
covid = full_covid.rename(columns={'submission_date': 'date', "tot_cases": "cases"})
covid.date = pd.to_datetime(covid.date)
covid = covid[covid.date < np.datetime64("2021-01-01")]
covid = covid[covid.date.dt.dayofweek == 0]
states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA',
       'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND',
       'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA',
       'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY']
covid = covid[np.isin(covid.state, states)]
covid = covid.pivot(index="date", columns="state")
covid = covid.diff()[1:]
covid.columns = [x[1] for x in covid.columns]

 Here are weekly COVID case counts across the US states plus DC, PR, and GU for 2020,
 a 48 x 53 matrix:

In [ ]:
covid

**Question:** Can we predict, say, Oregon's case counts
using the other states?

In [ ]:
from sklearn.linear_model import LinearRegression as lm
other_states = covid.loc[:,covid.columns != "OR"]
obs_OR = covid.loc[:,"OR"]
OR_fit = lm().fit(other_states, obs_OR)
est_OR = OR_fit.predict(other_states)

In [ ]:
plt.plot(covid.index, obs_OR, label="observed")
plt.plot(covid.index, est_OR, label="estimated", linestyle="--")
plt.legend(); plt.xlabel("date"); plt.ylabel("OR case counts");

Gee, we can predict the case counts *perfectly*?
Does that seem very likely? What's going on?

Let's think about what we're trying to do here.
We're trying to find coefficients $b_1, \ldots, b_k$
so that the the linear combination of the columns of $X$
$$  \hat y = b_1 X_{\cdot 1} + \cdots + b_k X_{\cdot k} $$
is as close to $y$ as possible.

Well, when is it possible to find $b$ so that $\hat y = y$?
It is possible if $y$ is in the column space of $X$.

Recall that $X$ is an $n \times k$ matrix.
If $n \le k$ then the columns of $X$ span then *entire* space $\mathbb{R}^n$
(unless for instance some columns are identical).

*Takeaway:* if you have more variables than observations
(the problem is *singular* and)
it is always possible to *exactly* predict the response.

*However,* these predictions are unlikely to be *generalizable*.

# Crossvalidation

How to tell how good your model is?

*See how well it predicts "new" data.*

To do $k$-fold crossvalidation:

1. Split your dataset into $k$ chunks (these should be independent!), and
2. for each chunk in turn, put it aside for "testing"
    and train your model on the remaining $k-1$ chunks.
3. Compare "test error" to "training error".

Predictions for data used to fit the model ("training error")
should not be much further off than for data held out ("test error").

This can be used either as an indication of *overfitting*
or to compare different models to each other.

# Crossvalidation set-up

Here's a pretty 'easy' prediction problem:

In [ ]:
n = 100
x = np.array([
    rng.uniform(low=0, high=10, size=n),
    rng.normal(size=n),
]).T
y = 2.5 + 1.5 * x[:,0] - 4.2 * x[:,1] + rng.normal(size=n)

fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.scatter(x[:,0], y); ax0.set_xlabel("x0"); ax0.set_ylabel("y")
ax1.scatter(x[:,1], y); ax1.set_xlabel("x1"); ax1.set_ylabel("y");

Let's (a) fit a linear model
and (b) do crossvalidation to look for evidence of overfitting.

In [ ]:
def rmse(X, y, model):
    yhat = model.predict(X)
    resids = y - yhat
    return np.sqrt(np.mean(resids ** 2))

def kfold(k, X, y, model):
    n  = len(y)
    folds = np.repeat(np.arange(k), np.ceil(n / k))[:n]
    rng.shuffle(folds)
    test_rmse = []
    train_rmse = []
    for ik in range(k):
        test_X = X[folds == ik]
        test_y = y[folds == ik]
        train_X = X[folds != ik]
        train_y = y[folds != ik]
        model.fit(train_X, train_y)
        test_rmse.append(rmse(test_X, test_y, model))
        train_rmse.append(rmse(train_X, train_y, model))
    return pd.DataFrame({
        "test" : test_rmse,
        "train" : train_rmse,
    })

crossval = kfold(5, x, y, lm())
crossval

*Conclusion:* test error was higher than training error,
but not much; there is (unsurprisingly) no evidence of (serious) overfitting.

# When you've got too many variables

We're going to *add more variables* -
these will be *independent* of everything else, so they should *not*
give us meaningful predictive power for $y$.
However, by chance each is a little correlated with $y$.

In [ ]:
crossval = pd.DataFrame()
for new_vars in np.linspace(0, 80, 9):
    new_x = rng.normal(size=(n, int(new_vars)))
    X = np.column_stack([x, new_x])
    xval = kfold(5, X, y, lm())
    xval["new_vars"] = int(new_vars)
    crossval = pd.concat([crossval, xval])

In [ ]:
crossval.groupby("new_vars").agg(np.mean).plot();

*Conclusion:* as the number of variables increases, the training error decreases - even though we *know* there's no new information being added!
The model is overfitting, which leads to increasing test (i.e., out-of-sample) error - the thing we generally care about.

# Regularization

Recall that, somewhat mysteriously,
scikit-learn's method to fit a Binomial GLM with a logistic link function
[has a "penalty" option](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
What's that?

Well, method finds $b$ to maximize the likelihood under the following model:
$$ Y_i \sim \text{Binomial}(N, p(X_i \cdot b)), $$
where $p(\cdot)$ is the logistic function.
The terms in the log-likelihood that depend on $b$ are
$$
    \sum_{i=1}^n \left\{
        Y_i \log(p(X_i \cdot b)) + (N_i - Y_i) \log(1 - p(X_i \cdot b))
    \right\} .
$$

The problem we had above
was that the variables that didn't matter
had small but nonzero estimated parameters;
and there were so many of them,
that together they added up to something big.

*Solution:* "encourage" them to be small.

So, with a "penalty" the method instead maximizes the log-likelihood **minus**
a "regularization" term that does the "encouraging".
Options:
$$\begin{aligned}
  \sum_j |b_j| \qquad & \text{"L1" or "$\ell_1$" or "lasso"}\\
  \sum_j b_j^2 \qquad & \text{"L2" or "$\ell_2$" or "ridge" or "Tikhonov"}
\end{aligned}$$

```
sklearn.linear_model.LogisticRegression(penalty="l1")
```
therefore finds the $b$ that maximizes
$$
    \sum_{i=1}^n \left\{
        Y_i \log(p(X_i \cdot b)) + (N_i - Y_i) \log(1 - p(X_i \cdot b))
    \right\} . - \sum_j |b_j| .
$$

# The Lasso

Let's try out this scheme on our example data, using [the Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html),
which fits a standard least-squares linear model but with a L1 penalty, minimizing
$$ \sum_i (y_i - X_i \cdot b)^2 + \alpha \sum_j |b_j| . $$

In [ ]:
from sklearn.linear_model import Lasso

ridge_crossval = pd.DataFrame()
for new_vars in np.linspace(0, 80, 9):
    new_x = rng.normal(size=(len(y), int(new_vars)))
    X = np.column_stack([x, new_x])
    xval = kfold(5, X, y, Lasso(alpha=.3))
    xval["new_vars"] = int(new_vars)
    ridge_crossval = pd.concat([ridge_crossval, xval])

In [ ]:
ridge_crossval.groupby("new_vars").agg(np.mean).plot();

Test error is still consistently higher than training error - as we'd expect - but only slightly.

# Exercise

Let's use *crossvalidation* to choose the *strength of regularization*
(i.e., the $\alpha$ parameter in the lasso).

We'll apply it to the `covid` data above.